In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/ETH/ETH HS24/Data Science Lab/legal_ner_mert/legal_ner"
! git pull
! ls

Mounted at /content/drive
/content/drive/MyDrive/ETH/ETH HS24/Data Science Lab/legal_ner_mert/legal_ner
Already up to date.
bge_data_exploration.ipynb	  finetune_anon.py		    __pycache__
check_anon_data.ipynb		  finetune_citation.py		    random_name_samplers.py
check_citation_data.ipynb	  finetune_data			    README.md
check_ood_data.ipynb		  finetune_generalist.py	    requirements.txt
check_pretrain_data.ipynb	  finetune_main.py		    synthetic_loc_generation.ipynb
chunk_anon_data.ipynb		  finetune_multi_merged_v1.py	    test_anon.ipynb
company_dataset_generation.ipynb  finetune_multi_merged_v2.py	    test_citation_extract.ipynb
config_synthetic_generation.yaml  finetune.py			    test_data_all
create_baseline_test_data.ipynb   generalist_data		    test_generalist.ipynb
create_citation_extract.ipynb	  main_anon_dataset_creation.ipynb  test_ood_anon.ipynb
create_generalist_data.ipynb	  merge_datasets.ipynb		    utils.py
create_test_datasets.ipynb	  models_generalist		    wandb
dataset_con

In [ ]:
!pip install gliner
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 75.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [ ]:
from gliner import GLiNER

import torch
import json
import os


from utils import get_all_labels
import random

In [ ]:
test_data_dir = f"test_data_all/"

# Load test sets

In [ ]:
with open(os.path.join(test_data_dir,'10_10_10.json'),'r') as f:
  test_v1 = json.load(f)

In [ ]:
test_ne = test_v1[:10000] + test_v1[-10000:]

In [ ]:
with open(os.path.join(test_data_dir,'urchadesynthetic-pii-ner-mistral-v1_test.json'),'r') as f:
  pii = json.load(f)
print(len(pii))

1964


In [ ]:
with open(os.path.join(test_data_dir,'pii_top_150.json'),'r') as f:
  pii_150 = json.load(f)
print(len(pii_150))

pii_labels_150 = get_all_labels(pii_150)

835


In [ ]:
with open(os.path.join(test_data_dir,'pii_top_100.json'),'r') as f:
  pii_100 = json.load(f)
print(len(pii_100))

673


# preparation

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

# merged_v1_gliner_multi-v2.1

## Checkpoint 150000

In [ ]:
model = GLiNER.from_pretrained("models_generalist/merged_v1_gliner_multi-v2.1/checkpoint-15000", load_tokenizer = True, local_files_only=True)
model = model.to(device)

config.json not found in /content/drive/MyDrive/ETH/ETH HS24/Data Science Lab/legal_ner_mert/legal_ner/models_generalist/merged_v1_gliner_multi-v2.1/checkpoint-15000


In [ ]:
results, f1 = model.evaluate(test_v1,flat_ner = True, batch_size = 12,threshold = 0.5,entity_types = ['person','organization','location','law','citation'])

In [ ]:
print(f"All 30K tests, including empty" : results)

P: 96.77%	R: 7.16%	F1: 13.33%
 0.13333470255393876


In [ ]:
random.seed(42)
results_ne, f1 = model.evaluate(random.sample(test_non_empty,1000),flat_ner = True, batch_size = 12,threshold = 0.001,entity_types = ['person','organization','location','law','citation'])

In [ ]:
print(results_ne)

P: 92.76%	R: 38.85%	F1: 54.76%



In [ ]:
random.seed(42)
results_thr, f1 = model.evaluate(random.sample(test_v1,1000),flat_ner = True, batch_size = 12,threshold = 0.001,entity_types = ['person','organization','location','law','citation'])

In [ ]:
print(results_thr)

P: 92.08%	R: 42.38%	F1: 58.05%



# merged_v2_gliner_multi-v2.1

In [ ]:
model = GLiNER.from_pretrained("models_generalist/merged_v2_gliner_multi-v2.1", load_tokenizer = True, local_files_only=True)
model = model.to(device)

config.json not found in /content/drive/MyDrive/ETH/ETH HS24/Data Science Lab/legal_ner_mert/legal_ner/models_generalist/merged_v2_gliner_multi-v2.1


In [ ]:
results, f1 = model.evaluate(test_v1,flat_ner = True, batch_size = 12,threshold = 0.5,entity_types = ['person','organization','location','law','citation'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
print(results)

P: 50.90%	R: 93.30%	F1: 65.87%



In [ ]:
results_ne, f1ne = model.evaluate(test_non_empty,flat_ner = True, batch_size = 12,threshold = 0.5,entity_types = ['person','organization','location','law','citation'])

In [ ]:
print(results_ne)

P: 69.79%	R: 93.30%	F1: 79.85%



In [ ]:
pii_labels = get_all_labels(pii)
len(pii_labels)

1386

In [ ]:
results_pii, f1pii = model.evaluate(pii,flat_ner = True, batch_size = 1,threshold = 0.5,entity_types = pii_labels[:100])
print(results_pii)

P: 50.02%	R: 52.19%	F1: 51.08%



In [ ]:
pii_labels_150 = get_all_labels(pii_150)
results_pii150, f1pii150 = model.evaluate(pii_150,flat_ner = True, batch_size = 1,threshold = 0.5,entity_types = pii_labels_150)
print(results_pii150)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


P: 43.54%	R: 38.00%	F1: 40.58%



baseline : gliner_multi-v2.1

In [ ]:
model_pii_baseline = GLiNER.from_pretrained('urchade/gliner_multi_pii-v1')
model_pii_baseline.to(device)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


GLiNER(
  (model): SpanModel(
    (token_rep_layer): Encoder(
      (bert_layer): Transformer(
        (model): DebertaV2Model(
          (embeddings): DebertaV2Embeddings(
            (word_embeddings): Embedding(250105, 768, padding_idx=0)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): StableDropout()
          )
          (encoder): DebertaV2Encoder(
            (layer): ModuleList(
              (0-11): 12 x DebertaV2Layer(
                (attention): DebertaV2Attention(
                  (self): DisentangledSelfAttention(
                    (query_proj): Linear(in_features=768, out_features=768, bias=True)
                    (key_proj): Linear(in_features=768, out_features=768, bias=True)
                    (value_proj): Linear(in_features=768, out_features=768, bias=True)
                    (pos_dropout): StableDropout()
                    (dropout): StableDropout()
                  )
                  (output): Debert

In [ ]:
model_pii_baseline.evaluate(pii_150,flat_ner = True, batch_size = 1,threshold = 0.5,entity_types = pii_labels_150)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


('P: 45.90%\tR: 60.72%\tF1: 52.28%\n', 0.5227788084930942)

In [ ]:
model_pii_baseline.evaluate(test_v1,flat_ner = True, batch_size = 12,threshold = 0.5,entity_types = ['person','organization','location','citation','law'])

('P: 14.43%\tR: 42.98%\tF1: 21.61%\n', 0.21605544005544006)

In [ ]:
#test the pii_baseline on the anon_non_empty only:
model_pii_baseline.evaluate(test_v1[:10000],flat_ner = True, batch_size = 12,threshold = 0.5,entity_types = ['person','organization','location'])

('P: 32.47%\tR: 65.81%\tF1: 43.49%\n', 0.43489629761692516)